<a href="https://colab.research.google.com/github/ProfOzpin/CMI-BFRB-Detection/blob/main/CNN_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

# Try to import cuML for GPU acceleration
try:
    import cudf
    import cuml
    from cuml.preprocessing import StandardScaler as cuMLStandardScaler
    from cuml.model_selection import train_test_split as cuml_train_test_split
    CUML_AVAILABLE = True
    print("cuML available - GPU acceleration enabled")
except ImportError:
    CUML_AVAILABLE = False
    print("cuML not available - using CPU preprocessing")

# Set mixed precision for GPU optimization
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# Constants for the model
MAX_SEQUENCE_LENGTH = 50
NUM_IMU_FEATURES = 7
NUM_THERMOPILE = 5
NUM_TOF_SENSORS = 5
TOF_PIXELS_PER_SENSOR = 64
NUM_DEMOGRAPHIC_FEATURES = 7

# Custom Focal Loss for handling class imbalance
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=0.25, gamma=2.0, name='focal_loss'):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)

        # Clip predictions to prevent log(0)
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Calculate focal loss
        alpha_factor = y_true * self.alpha + (1 - y_true) * (1 - self.alpha)
        focal_weight = y_true * (1 - y_pred) ** self.gamma + (1 - y_true) * y_pred ** self.gamma

        ce_loss = -y_true * tf.math.log(y_pred) - (1 - y_true) * tf.math.log(1 - y_pred)
        focal_loss = alpha_factor * focal_weight * ce_loss

        return tf.reduce_mean(focal_loss)

# Custom Sparse Categorical Focal Loss for multiclass
class SparseCategoricalFocalLoss(tf.keras.losses.Loss):
    def __init__(self, alpha=1.0, gamma=2.0, name='sparse_categorical_focal_loss'):
        super().__init__(name=name)
        self.alpha = alpha
        self.gamma = gamma

    def call(self, y_true, y_pred):
        y_true = tf.cast(y_true, tf.int32)
        y_pred = tf.cast(y_pred, tf.float32)

        # Convert sparse labels to one-hot
        num_classes = tf.shape(y_pred)[-1]
        y_true_one_hot = tf.one_hot(y_true, num_classes)

        # Clip predictions
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)

        # Calculate focal loss
        ce_loss = -tf.reduce_sum(y_true_one_hot * tf.math.log(y_pred), axis=-1)
        p_t = tf.reduce_sum(y_true_one_hot * y_pred, axis=-1)
        focal_weight = self.alpha * (1 - p_t) ** self.gamma

        focal_loss = focal_weight * ce_loss
        return tf.reduce_mean(focal_loss)

# Time Series Data Augmentation Functions
def time_series_augmentation(data, augment_prob=0.5):
    """Apply various time series augmentation techniques"""
    if np.random.random() < augment_prob:
        augmentation_type = np.random.choice(['jitter', 'scale', 'time_warp', 'magnitude_warp'])

        if augmentation_type == 'jitter':
            # Add random noise
            noise = np.random.normal(0, 0.03, data.shape)
            return data + noise

        elif augmentation_type == 'scale':
            # Scale the magnitude
            scale_factor = np.random.uniform(0.8, 1.2)
            return data * scale_factor

        elif augmentation_type == 'time_warp':
            # Time warping by random sampling
            seq_len = data.shape[0]
            indices = np.sort(np.random.choice(seq_len, int(seq_len * 0.9), replace=False))
            warped_data = np.zeros_like(data)
            warped_indices = np.linspace(0, seq_len-1, len(indices)).astype(int)
            warped_data[warped_indices] = data[indices]
            return warped_data

        elif augmentation_type == 'magnitude_warp':
            # Smooth magnitude warping
            warp = np.random.normal(1.0, 0.1, data.shape[1])
            return data * warp

    return data

def load_and_preprocess_data(train_file, train_demographics_file, test_file=None, test_demographics_file=None):
    """data loading with class balance analysis"""
    print("Loading data...")

    if CUML_AVAILABLE:
        train_df = cudf.read_csv(train_file)
        train_demo_df = cudf.read_csv(train_demographics_file)
    else:
        train_df = pd.read_csv(train_file)
        train_demo_df = pd.read_csv(train_demographics_file)

    print("Processing training sequences...")
    train_sequences = process_sequences(train_df, train_demo_df)

    print("Preparing training data for model...")
    X_train, y_train = prepare_model_data(train_sequences)

    # Analyze class distribution
    binary_counts = np.bincount(y_train['binary_output'])
    print(f"Binary class distribution: {binary_counts}")
    print(f"Class imbalance ratio: {binary_counts[0]/binary_counts[1] if len(binary_counts) > 1 else 'No positive samples'}")

    # Calculate class weights for imbalanced data
    global BINARY_CLASS_WEIGHTS, MULTICLASS_CLASS_WEIGHTS

    if len(binary_counts) > 1:
        BINARY_CLASS_WEIGHTS = compute_class_weight(
            'balanced',
            classes=np.unique(y_train['binary_output']),
            y=y_train['binary_output']
        )
    else:
        BINARY_CLASS_WEIGHTS = np.array([1.0, 1.0])

    MULTICLASS_CLASS_WEIGHTS = compute_class_weight(
        'balanced',
        classes=np.unique(y_train['multiclass_output']),
        y=y_train['multiclass_output']
    )

    print(f"Binary class weights: {BINARY_CLASS_WEIGHTS}")
    print(f"Multiclass class weights shape: {MULTICLASS_CLASS_WEIGHTS.shape}")

    # Create train/validation split with stratification
    train_indices, val_indices = train_test_split(
        range(len(X_train['imu_input'])),
        test_size=0.2,
        random_state=42,
        stratify=y_train['binary_output']  # Stratify by binary target
    )

    X_train_split = {key: value[train_indices] for key, value in X_train.items()}
    X_val = {key: value[val_indices] for key, value in X_train.items()}
    y_train_split = {key: value[train_indices] for key, value in y_train.items()}
    y_val = {key: value[val_indices] for key, value in y_train.items()}

    data_dict = {
        'X_train': X_train_split,
        'y_train': y_train_split,
        'X_val': X_val,
        'y_val': y_val
    }

    # Load test data if provided
    if test_file and test_demographics_file:
        print("Loading test data...")
        if CUML_AVAILABLE:
            test_df = cudf.read_csv(test_file)
            test_demo_df = cudf.read_csv(test_demographics_file)
        else:
            test_df = pd.read_csv(test_file)
            test_demo_df = pd.read_csv(test_demographics_file)

        print("Processing test sequences...")
        test_sequences = process_sequences(test_df, test_demo_df, is_train=False)

        print("Preparing test data for model...")
        X_test, sequence_ids = prepare_model_data(test_sequences, is_train=False)

        data_dict['X_test'] = X_test
        data_dict['sequence_ids'] = sequence_ids

    return data_dict

def process_sequences(df, demo_df, is_train=True):
    """sequence processing with better error handling"""
    sequences = []

    if CUML_AVAILABLE:
        sequence_ids = df['sequence_id'].unique().values_host
    else:
        sequence_ids = df['sequence_id'].unique()

    for seq_id in sequence_ids:
        seq_data = df[df['sequence_id'] == seq_id]

        if len(seq_data) == 0:
            print(f"Warning: Empty sequence data for sequence ID {seq_id}")
            continue

        if CUML_AVAILABLE:
            seq_data = seq_data.to_pandas()
            demo_df_pandas = demo_df.to_pandas()
        else:
            demo_df_pandas = demo_df

        subject_id = seq_data['subject'].iloc[0]
        subject_demo = demo_df_pandas[demo_df_pandas['subject'] == subject_id]

        if len(subject_demo) == 0:
            print(f"Warning: No demographic data found for subject {subject_id}")
            # Use default demographic values
            demo_features = [0.0, 25.0, 0.0, 1.0, 170.0, 52.0, 28.0]
        else:
            demo_features = [
                subject_demo['adult_child'].fillna(0).iloc[0],
                subject_demo['age'].fillna(25).iloc[0],
                subject_demo['sex'].fillna(0).iloc[0],
                subject_demo['handedness'].fillna(1).iloc[0],
                subject_demo['height_cm'].fillna(170).iloc[0],
                subject_demo['shoulder_to_wrist_cm'].fillna(52).iloc[0],
                subject_demo['elbow_to_wrist_cm'].fillna(28).iloc[0]
            ]

        # Extract sensor columns
        imu_cols = [col for col in seq_data.columns if col.startswith('acc_') or col.startswith('rot_')]
        thm_cols = [col for col in seq_data.columns if col.startswith('thm_')]
        tof_cols = [col for col in seq_data.columns if col.startswith('tof_')]

        # Handle sequence length with padding/truncation
        if len(seq_data) < MAX_SEQUENCE_LENGTH:
            padding_needed = MAX_SEQUENCE_LENGTH - len(seq_data)
            last_row = seq_data.iloc[-1:].copy()
            for _ in range(padding_needed):
                seq_data = pd.concat([seq_data, last_row])
        elif len(seq_data) > MAX_SEQUENCE_LENGTH:
            seq_data = seq_data.iloc[:MAX_SEQUENCE_LENGTH]

        # Extract and process sensor data
        imu_data = seq_data[imu_cols].fillna(0).values
        thm_data = seq_data[thm_cols].fillna(0).values
        tof_data = seq_data[tof_cols].fillna(0).values

        # Apply data augmentation for training data
        if is_train:
            imu_data = time_series_augmentation(imu_data)
            thm_data = time_series_augmentation(thm_data)

        # Create ToF mask and normalize
        tof_mask = (tof_data != -1).astype(np.float32)
        tof_data = np.where(tof_data == -1, 0, tof_data)

        sequence = {
            'sequence_id': seq_id,
            'imu_data': imu_data,
            'thm_data': thm_data,
            'tof_data': tof_data,
            'tof_mask': tof_mask,
            'demographic': demo_features,
        }

        if is_train:
            sequence_type = seq_data['sequence_type'].iloc[0]
            gesture = seq_data['gesture'].iloc[0]
            binary_target = 1 if sequence_type == 'Target' else 0

            sequence['binary_target'] = binary_target
            sequence['gesture'] = gesture

        sequences.append(sequence)

    return sequences

def prepare_model_data(sequences, is_train=True):
    """data preparation with improved normalization"""
    imu_data = []
    thm_data = []
    tof_data = []
    tof_mask = []
    demo_data = []
    binary_targets = []
    gesture_targets = []
    sequence_ids = []

    for seq in sequences:
        imu_data.append(seq['imu_data'])
        thm_data.append(seq['thm_data'])
        tof_data.append(seq['tof_data'])
        tof_mask.append(seq['tof_mask'])
        demo_data.append(seq['demographic'])
        sequence_ids.append(seq['sequence_id'])

        if is_train:
            binary_targets.append(seq['binary_target'])
            gesture_targets.append(seq['gesture'])

    # Convert CuPy arrays to NumPy
    def cupy_to_numpy(data):
        if hasattr(data, 'get'):
            return data.get()
        elif isinstance(data, list):
            return [cupy_to_numpy(item) for item in data]
        else:
            return data

    imu_data = [cupy_to_numpy(item) for item in imu_data]
    thm_data = [cupy_to_numpy(item) for item in thm_data]
    tof_data = [cupy_to_numpy(item) for item in tof_data]
    tof_mask = [cupy_to_numpy(item) for item in tof_mask]
    demo_data = [cupy_to_numpy(item) for item in demo_data]

    # Convert to numpy arrays
    imu_data = np.array(imu_data)
    thm_data = np.array(thm_data)
    tof_data = np.array(tof_data)
    tof_mask = np.array(tof_mask)
    demo_data = np.array(demo_data)

    # normalization
    from sklearn.preprocessing import StandardScaler as SKStandardScaler, RobustScaler

    # Use RobustScaler for better outlier handling
    imu_scaler = RobustScaler()
    thm_scaler = RobustScaler()
    demo_scaler = SKStandardScaler()

    # Normalize sensor data
    imu_shape = imu_data.shape
    thm_shape = thm_data.shape

    imu_flat = imu_data.reshape(-1, imu_shape[2])
    thm_flat = thm_data.reshape(-1, thm_shape[2])

    # Fit and transform with outlier-robust scaling
    imu_flat = imu_scaler.fit_transform(imu_flat)
    thm_flat = thm_scaler.fit_transform(thm_flat)
    demo_data = demo_scaler.fit_transform(demo_data)

    imu_data = imu_flat.reshape(imu_shape)
    thm_data = thm_flat.reshape(thm_shape)

    # Improved ToF normalization
    tof_data = np.where(tof_mask == 1, tof_data / 254.0, 0)

    X = {
        'imu_input': imu_data,
        'thm_input': thm_data,
        'tof_input': tof_data,
        'tof_mask': tof_mask,
        'demo_input': demo_data
    }

    if is_train:
        binary_targets = cupy_to_numpy(binary_targets)
        gesture_targets = cupy_to_numpy(gesture_targets)

        binary_targets = np.array(binary_targets)

        label_encoder = LabelEncoder()
        gesture_targets = label_encoder.fit_transform(gesture_targets)
        gesture_targets = np.array(gesture_targets)

        y = {
            'binary_output': binary_targets,
            'multiclass_output': gesture_targets
        }

        return X, y
    else:
        return X, sequence_ids

def build_multimodal_cnn(
    sequence_length=MAX_SEQUENCE_LENGTH,
    num_imu_features=NUM_IMU_FEATURES,
    num_thermopile=NUM_THERMOPILE,
    num_tof_sensors=NUM_TOF_SENSORS,
    tof_pixels=TOF_PIXELS_PER_SENSOR,
    num_demographic=NUM_DEMOGRAPHIC_FEATURES,
    num_gestures=19
):
    """CNN architecture with better regularization and fusion"""

    # Input layers
    imu_input = Input(shape=(sequence_length, num_imu_features), name='imu_input')
    thm_input = Input(shape=(sequence_length, num_thermopile), name='thm_input')
    tof_input = Input(shape=(sequence_length, num_tof_sensors * tof_pixels), name='tof_input')
    tof_mask = Input(shape=(sequence_length, num_tof_sensors * tof_pixels), name='tof_mask')
    demo_input = Input(shape=(num_demographic,), name='demo_input')

    # IMU branch with residual connections
    x_imu = layers.Conv1D(64, 5, activation='relu', padding='same')(imu_input)
    x_imu = layers.BatchNormalization()(x_imu)
    x_imu = layers.Dropout(0.2)(x_imu)

    # Residual block
    imu_residual = layers.Conv1D(64, 3, activation='relu', padding='same')(x_imu)
    imu_residual = layers.BatchNormalization()(imu_residual)
    x_imu = layers.Add()([x_imu, imu_residual])

    x_imu = layers.Conv1D(128, 3, activation='relu', padding='same')(x_imu)
    x_imu = layers.BatchNormalization()(x_imu)
    x_imu = layers.MaxPooling1D(2)(x_imu)
    x_imu = layers.Dropout(0.3)(x_imu)

    x_imu = layers.Conv1D(256, 3, activation='relu', padding='same')(x_imu)
    x_imu = layers.BatchNormalization()(x_imu)
    x_imu = layers.GlobalAveragePooling1D()(x_imu)
    x_imu = layers.Dropout(0.4)(x_imu)

    # Thermopile branch
    x_thm = layers.Conv1D(32, 3, activation='relu', padding='same')(thm_input)
    x_thm = layers.BatchNormalization()(x_thm)
    x_thm = layers.Dropout(0.2)(x_thm)

    x_thm = layers.Conv1D(64, 3, activation='relu', padding='same')(x_thm)
    x_thm = layers.BatchNormalization()(x_thm)
    x_thm = layers.MaxPooling1D(2)(x_thm)
    x_thm = layers.Dropout(0.3)(x_thm)

    x_thm = layers.GlobalAveragePooling1D()(x_thm)
    x_thm = layers.Dropout(0.4)(x_thm)

    # ToF branch with attention mechanism
    x_tof = layers.Reshape((sequence_length, num_tof_sensors, 8, 8))(tof_input)
    mask_reshaped = layers.Reshape((sequence_length, num_tof_sensors, 8, 8))(tof_mask)
    x_tof = layers.Multiply()([x_tof, mask_reshaped])

    x_tof = layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(x_tof)

    # 3D CNN with attention
    x_tof = layers.TimeDistributed(
        layers.Conv3D(32, (1, 3, 3), activation='relu', padding='same')
    )(x_tof)
    x_tof = layers.TimeDistributed(layers.BatchNormalization())(x_tof)
    x_tof = layers.TimeDistributed(layers.MaxPooling3D(pool_size=(1, 2, 2)))(x_tof)

    x_tof = layers.TimeDistributed(
        layers.Conv3D(64, (1, 3, 3), activation='relu', padding='same')
    )(x_tof)
    x_tof = layers.TimeDistributed(layers.BatchNormalization())(x_tof)

    # Flatten and apply 1D convolution
    x_tof = layers.TimeDistributed(layers.Reshape((-1,)))(x_tof)
    x_tof = layers.Conv1D(128, 3, activation='relu', padding='same')(x_tof)
    x_tof = layers.BatchNormalization()(x_tof)
    x_tof = layers.GlobalAveragePooling1D()(x_tof)
    x_tof = layers.Dropout(0.4)(x_tof)

    # demographic branch
    x_demo = layers.Dense(64, activation='relu')(demo_input)
    x_demo = layers.BatchNormalization()(x_demo)
    x_demo = layers.Dropout(0.3)(x_demo)

    x_demo = layers.Dense(32, activation='relu')(x_demo)
    x_demo = layers.BatchNormalization()(x_demo)
    x_demo = layers.Dropout(0.3)(x_demo)

    # Adaptive weighted fusion with attention
    tof_availability = layers.Lambda(lambda x: tf.reduce_mean(x, axis=[1, 2]))(tof_mask)
    tof_weight = layers.Lambda(lambda x: tf.nn.sigmoid(x * 5))(tof_availability)  # Sharper weighting

    # Apply adaptive weighting
    tof_weight_expanded = layers.Reshape((1,))(tof_weight)
    x_tof_weighted = layers.Multiply()([x_tof, tof_weight_expanded])

    # Attention mechanism for feature fusion
    concat_features = layers.Concatenate()([x_imu, x_thm, x_tof_weighted, x_demo])

    # Attention weights
    attention = layers.Dense(concat_features.shape[-1], activation='sigmoid')(concat_features)
    attended_features = layers.Multiply()([concat_features, attention])

    # shared layers with regularization
    shared = layers.Dense(512, activation='relu')(attended_features)
    shared = layers.BatchNormalization()(shared)
    shared = layers.Dropout(0.5)(shared)

    shared = layers.Dense(256, activation='relu')(shared)
    shared = layers.BatchNormalization()(shared)
    shared = layers.Dropout(0.5)(shared)

    shared = layers.Dense(128, activation='relu')(shared)
    shared = layers.BatchNormalization()(shared)
    shared = layers.Dropout(0.4)(shared)

    # Output layers
    binary_output = layers.Dense(1, activation='sigmoid', name='binary_output')(shared)
    multiclass_output = layers.Dense(num_gestures, activation='softmax', name='multiclass_output')(shared)

    model = Model(
        inputs=[imu_input, thm_input, tof_input, tof_mask, demo_input],
        outputs=[binary_output, multiclass_output]
    )

    return model

# Custom Metrics
class WeightedF1Score(tf.keras.metrics.Metric):
    def __init__(self, name='weighted_f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_binary = tf.cast(tf.greater_equal(y_pred, 0.5), tf.float32)
        self.precision.update_state(y_true, y_pred_binary, sample_weight)
        self.recall.update_state(y_true, y_pred_binary, sample_weight)

    def result(self):
        p = self.precision.result()
        r = self.recall.result()
        return 2 * ((p * r) / (p + r + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

class MacroF1Score(tf.keras.metrics.Metric):
    def __init__(self, num_classes, name='macro_f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.true_positives = self.add_weight(
            shape=(num_classes,), name='true_positives', initializer='zeros'
        )
        self.false_positives = self.add_weight(
            shape=(num_classes,), name='false_positives', initializer='zeros'
        )
        self.false_negatives = self.add_weight(
            shape=(num_classes,), name='false_negatives', initializer='zeros'
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred_classes = tf.argmax(y_pred, axis=1)
        y_pred_one_hot = tf.one_hot(y_pred_classes, self.num_classes)

        if len(y_true.shape) == 1 or y_true.shape[1] == 1:
            y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), self.num_classes)
        else:
            y_true_one_hot = y_true

        tp = tf.reduce_sum(y_true_one_hot * y_pred_one_hot, axis=0)
        fp = tf.reduce_sum((1 - y_true_one_hot) * y_pred_one_hot, axis=0)
        fn = tf.reduce_sum(y_true_one_hot * (1 - y_pred_one_hot), axis=0)

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1_per_class = 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))
        return tf.reduce_mean(f1_per_class)

    def reset_states(self):
        self.true_positives.assign(tf.zeros_like(self.true_positives))
        self.false_positives.assign(tf.zeros_like(self.false_positives))
        self.false_negatives.assign(tf.zeros_like(self.false_negatives))

def train_model(model, data, epochs=100, batch_size=32, output_dir='models'):
    """training with advanced callbacks and monitoring"""
    os.makedirs(output_dir, exist_ok=True)

    # callbacks with better scheduling
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,  # Increased patience
            restore_best_weights=True,
            verbose=1,
            min_delta=0.001
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.3,  # More aggressive reduction
            patience=7,   # Shorter patience for LR reduction
            min_lr=1e-7,
            verbose=1,
            cooldown=3
        ),
        ModelCheckpoint(
            filepath=os.path.join(output_dir, 'best_model.keras'),
            monitor='val_loss',
            save_best_only=True,
            verbose=1,
            save_weights_only=False
        ),
        # Learning rate warmup
        tf.keras.callbacks.LearningRateScheduler(
            lambda epoch: 1e-5 if epoch < 3 else 1e-3 * (0.95 ** (epoch - 3))
        )
    ]

    print("Starting model training...")
    history = model.fit(
        x=data['X_train'],
        y=data['y_train'],
        batch_size=batch_size,
        epochs=epochs,
        validation_data=(data['X_val'], data['y_val']),
        callbacks=callbacks,
        verbose=1,
        shuffle=True
    )

    # Save final model and history
    model.save(os.path.join(output_dir, 'final_model.keras'))
    pd.DataFrame(history.history).to_csv(
        os.path.join(output_dir, 'training_history.csv'), index=False
    )

    print(f"model training complete. Models saved to {output_dir}")
    return model, history

def evaluate_model(model, data):
    """evaluation with detailed metrics"""
    print("Evaluating model...")

    results = model.evaluate(data['X_val'], data['y_val'], verbose=1)

    metrics = {}
    for i, metric_name in enumerate(model.metrics_names):
        metrics[metric_name] = results[i]

    # Additional detailed analysis
    predictions = model.predict(data['X_val'])
    binary_preds = predictions[0]
    multiclass_preds = predictions[1]

    # Binary classification analysis
    binary_pred_classes = (binary_preds > 0.5).astype(int).flatten()
    y_true_binary = data['y_val']['binary_output']

    from sklearn.metrics import classification_report, confusion_matrix
    print("\nBinary Classification Report:")
    print(classification_report(y_true_binary, binary_pred_classes))
    print("\nBinary Confusion Matrix:")
    print(confusion_matrix(y_true_binary, binary_pred_classes))

    # Multiclass analysis
    multiclass_pred_classes = np.argmax(multiclass_preds, axis=1)
    y_true_multiclass = data['y_val']['multiclass_output']

    print("\nMulticlass Classification Report:")
    print(classification_report(y_true_multiclass, multiclass_pred_classes))

    print("\nEvaluation metrics:")
    for name, value in metrics.items():
        print(f"{name}: {value:.4f}")

    return metrics

def predict_and_save(model, data, output_file):
    """prediction with confidence scores"""
    print("Generating predictions...")

    predictions = model.predict(data['X_test'])
    binary_preds = predictions[0]
    multiclass_preds = predictions[1]

    # prediction processing
    binary_classes = (binary_preds > 0.5).astype(int).flatten()
    binary_confidence = np.max([binary_preds, 1-binary_preds], axis=0).flatten()

    multiclass_classes = np.argmax(multiclass_preds, axis=1)
    multiclass_confidence = np.max(multiclass_preds, axis=1)

    # gesture mapping
    gesture_map = {
        0: "non_target",
        1: "above_ear_pull_hair",
        2: "forehead_pull_hairline",
        3: "forehead_scratch",
        4: "eyebrow_pull_hair",
        5: "eyelash_pull_hair",
        6: "neck_pinch_skin",
        7: "neck_scratch",
        8: "cheek_pinch_skin",
        9: "drink_from_bottle",
        10: "glasses_on_off",
        11: "pull_air_toward_face",
        12: "pinch_knee_leg_skin",
        13: "scratch_knee_leg_skin",
        14: "write_name_on_leg",
        15: "text_on_phone",
        16: "feel_around_tray",
        17: "write_name_in_air",
        18: "wave_hello"
    }

    output_df = pd.DataFrame({
        'sequence_id': data['sequence_ids'],
        'is_target': binary_classes,
        'binary_confidence': binary_confidence,
        'gesture_class': multiclass_classes,
        'multiclass_confidence': multiclass_confidence,
        'gesture': [gesture_map.get(cls, f"gesture_{cls}") for cls in multiclass_classes]
    })

    output_df.to_csv(output_file, index=False)
    print(f"predictions saved to {output_file}")

def save_model(model, model_dir):
    """model saving with metadata"""
    os.makedirs(model_dir, exist_ok=True)

    # Save in multiple formats
    model.save(os.path.join(model_dir, 'model.keras'))
    model.export(os.path.join(model_dir, 'savedmodel'))
    model.save(os.path.join(model_dir, 'model.h5'))

    # Save architecture and weights separately
    model_json = model.to_json()
    with open(os.path.join(model_dir, 'model_architecture.json'), 'w') as f:
        f.write(model_json)

    model.save_weights(os.path.join(model_dir, 'model.weights.h5'))

    # Save model metadata
    metadata = {
        'model_type': 'Multimodal_BFRB_CNN',
        'input_shape': {
            'imu': (MAX_SEQUENCE_LENGTH, NUM_IMU_FEATURES),
            'thermopile': (MAX_SEQUENCE_LENGTH, NUM_THERMOPILE),
            'tof': (MAX_SEQUENCE_LENGTH, NUM_TOF_SENSORS * TOF_PIXELS_PER_SENSOR),
            'demographic': (NUM_DEMOGRAPHIC_FEATURES,)
        },
        'num_classes': 19,
        'improvements': [
            'focal_loss', 'data_augmentation', 'class_weights',
            'architecture', 'attention_mechanism'
        ]
    }

    import json
    with open(os.path.join(model_dir, 'model_metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"model saved to {model_dir} with metadata")

def load_model(model_dir):
    """model loading with format fallback"""
    try:
        model = tf.keras.models.load_model(
            os.path.join(model_dir, 'model.keras'),
            custom_objects={
                'FocalLoss': FocalLoss,
                'SparseCategoricalFocalLoss': SparseCategoricalFocalLoss,
                'WeightedF1Score': WeightedF1Score,
                'MacroF1Score': MacroF1Score
            }
        )
        print(f"Loaded model from {model_dir}")
    except:
        try:
            model = tf.keras.models.load_model(
                os.path.join(model_dir, 'model.h5'),
                custom_objects={
                    'FocalLoss': FocalLoss,
                    'SparseCategoricalFocalLoss': SparseCategoricalFocalLoss,
                    'WeightedF1Score': WeightedF1Score,
                    'MacroF1Score': MacroF1Score
                }
            )
            print(f"Loaded model from H5 format: {model_dir}")
        except:
            # Load from architecture and weights
            with open(os.path.join(model_dir, 'model_architecture.json'), 'r') as f:
                model_json = f.read()

            model = tf.keras.models.model_from_json(model_json)
            model.load_weights(os.path.join(model_dir, 'model.weights.h5'))
            print(f"Loaded model from architecture + weights: {model_dir}")

    return model

def main():
    """main execution with comprehensive improvements"""

    # File paths
    train_file = 'train.csv'
    train_demo_file = 'train_demographics.csv'
    test_file = 'test.csv'
    test_demo_file = 'test_demographics.csv'

    # Load and preprocess data with enhancements
    data = load_and_preprocess_data(train_file, train_demo_file, test_file, test_demo_file)

    # Build model
    model = build_multimodal_cnn()

    # Compile with focal loss and class weights
    model.compile(
        optimizer=Adam(learning_rate=1e-3, clipnorm=1.0),  # Gradient clipping
        loss={
            'binary_output': FocalLoss(alpha=BINARY_CLASS_WEIGHTS[1]/BINARY_CLASS_WEIGHTS[0], gamma=2.0),
            'multiclass_output': SparseCategoricalFocalLoss(alpha=1.0, gamma=2.0)
        },
        metrics={
            'binary_output': [WeightedF1Score(), 'accuracy'],
            'multiclass_output': ['accuracy', MacroF1Score(num_classes=19)]
        },
        loss_weights={
            'binary_output': 0.6,  # Higher weight for binary task
            'multiclass_output': 0.4
        }
    )

    print("\nModel Summary:")
    model.summary()

    # Train with configuration
    if 'X_train' in data and 'y_train' in data:
        model, history = train_model(
            model, data, epochs=100, batch_size=16,  # Smaller batch for better convergence
            output_dir='bfrb_model'
        )

        # evaluation
        if 'X_val' in data and 'y_val' in data:
            metrics = evaluate_model(model, data)

    # Generate predictions
    if 'X_test' in data:
        predict_and_save(model, data, 'bfrb_predictions.csv')

    # Save model
    save_model(model, 'bfrb_model')

    print("BFRB detection pipeline complete with all improvements")

if __name__ == "__main__":
    main()


cuML available - GPU acceleration enabled
Loading data...
Processing training sequences...
Preparing training data for model...
Binary class distribution: [3038 5113]
Class imbalance ratio: 0.5941717191472716
Binary class weights: [1.34150757 0.79708586]
Multiclass class weights shape: (18,)
Loading test data...
Processing test sequences...
Preparing test data for model...

Enhanced Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ imu_input           │ (None, 50, 7)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tof_input           │ (None, 50, 320)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ tof_mask            │ (None, 50, 320)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_6 (Cast)       │ (None, 50, 7)     │          0 │ imu_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_8 (Cast)       │ (None, 50, 320)   │          0 │ tof_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_9 (Cast)       │ (None, 50, 320)   │          0 │ tof_mask[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 50, 64)    │      2,304 │ cast_6[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_4 (Reshape) │ (None, 50, 5, 8,  │          0 │ cast_8[0][0]      │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_5 (Reshape) │ (None, 50, 5, 8,  │          0 │ cast_9[0][0]      │
│                     │ 8)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 64)    │        256 │ conv1d_7[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_3          │ (None, 50, 5, 8,  │          0 │ reshape_4[0][0],  │
│ (Multiply)          │ 8)                │            │ reshape_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_12          │ (None, 50, 64)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda_3 (Lambda)   │ (None, 50, 5, 8,  │          0 │ multiply_3[0][0]  │
│                     │ 8, 1)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 50, 64)    │     12,352 │ dropout_12[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ thm_input           │ (None, 50, 5)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_6  │ (None, 50, 5, 8,  │        320 │ lambda_3[0][0]    │
│ (TimeDistributed)   │ 8, 32)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 50, 64)    │        256 │ conv1d_8[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ cast_7 (Cast)       │ (None, 50, 5)     │          0 │ thm_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_distributed_7  │ (None, 50, 5, 8,  │        128 │ time_distributed

 Total params: 2,783,508 (10.62 MB)

 Trainable params: 2,779,860 (10.60 MB)

 Non-trainable params: 3,648 (14.25 KB)

Starting enhanced model training...
Epoch 1/100
408/408 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step - binary_output_accuracy: 0.4902 - binary_output_loss: 0.3367 - binary_output_weighted_f1_score: 0.5491 - loss: 1.7723 - multiclass_output_accuracy: 0.0511 - multiclass_output_loss: 3.9256 - multiclass_output_macro_f1_score: 0.0461
Epoch 1: val_loss improved from inf to 1.17869, saving model to enhanced_bfrb_model/best_model.keras
408/408 ━━━━━━━━━━━━━━━━━━━━ 298s 364ms/step - binary_output_accuracy: 0.4902 - binary_output_loss: 0.3367 - binary_output_weighted_f1_score: 0.5492 - loss: 1.7722 - multiclass_output_accuracy: 0.0511 - multiclass_output_loss: 3.9255 - multiclass_output_macro_f1_score: 0.0461 - val_binary_output_accuracy: 0.5543 - val_binary_output_loss: 0.1054 - val_binary_output_weighted_f1_score: 0.6851 - val_loss: 1.1787 - val_multiclass_output_accuracy: 0.0846 - val_multiclass_output_loss: 2.7887 - val_multiclass_output_macro_f1_score: 0.0331 - learning_rate: 1.0000e-05
Epoch 2/100

In [7]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c cmi-detect-behavior-with-sensor-data

mkdir: cannot create directory ‘/root/.kaggle’: File exists
 71% 127M/178M [00:00<00:00, 1.32GB/s]
100% 178M/178M [00:00<00:00, 1.27GB/s]


In [8]:
import zipfile
with zipfile.ZipFile("cmi-detect-behavior-with-sensor-data.zip", 'r') as zip_ref:
    zip_ref.extractall("./")